In [1]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

import warnings

warnings.filterwarnings('ignore')

In [2]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [3]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [4]:
df_train['Segmentation'].value_counts()

D    2268
A    1972
C    1970
B    1858
Name: Segmentation, dtype: int64

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [6]:
x=df_train[df_train['Work_Experience']==0]

In [7]:
x[x['Profession']=='Artist']

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
5,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,C
19,464291,Female,Yes,79,Yes,Artist,0.0,High,1.0,Cat_6,C
28,465882,Male,No,35,Yes,Artist,0.0,Low,1.0,Cat_6,B
30,463331,Female,Yes,45,Yes,Artist,0.0,Low,3.0,Cat_6,C
46,465729,Male,Yes,41,Yes,Artist,0.0,Average,4.0,Cat_4,B
...,...,...,...,...,...,...,...,...,...,...,...
8024,467153,Female,Yes,51,Yes,Artist,0.0,Low,NaN,Cat_6,A
8050,466004,Male,Yes,52,Yes,Artist,0.0,Average,6.0,Cat_4,C
8060,463613,Female,Yes,48,Yes,Artist,0.0,Average,6.0,Cat_6,A
8061,465231,Male,Yes,65,No,Artist,0.0,Average,2.0,Cat_6,C


In [8]:
map_gender={"Male" : 1, "Female": 2}
df_train['Gender'] = df_train['Gender'].map(map_gender)
df_test['Gender'] = df_test['Gender'].map(map_gender)


In [9]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,1,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,2,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,2,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,1,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,2,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [10]:
df_train.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 

In [11]:
map_married={"No" : 1, "Yes": 2}
df_train['Ever_Married'] = df_train['Ever_Married'].map(map_married)
df_test['Ever_Married'] = df_test['Ever_Married'].map(map_married)

In [12]:
df_train["Ever_Married"].fillna( method ='ffill', inplace = True) 
df_test["Ever_Married"].fillna( method ='ffill', inplace = True) 

In [13]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
4321,461253,2,1.0,18,No,Healthcare,0.0,Low,5.0,Cat_4,D
3992,466750,2,1.0,18,NaN,Healthcare,0.0,Low,3.0,Cat_3,D
1155,466814,1,1.0,18,No,Healthcare,0.0,Low,3.0,Cat_6,D
1159,467474,1,1.0,18,No,Healthcare,3.0,Low,5.0,Cat_6,D
5827,462764,1,1.0,18,No,Healthcare,NaN,Low,4.0,Cat_6,D


In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8068 entries, 4321 to 7262
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   int64  
 2   Ever_Married     8068 non-null   float64
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(3), int64(3), object(5)
memory usage: 756.4+ KB


In [15]:
df_train['Spending_Score'].value_counts()

Low        4878
Average    1974
High       1216
Name: Spending_Score, dtype: int64

In [16]:
map_score={"Low" : 1, "Average": 2,"High":3}
df_train['Spending_Score'] = df_train['Spending_Score'].map(map_score)
df_test['Spending_Score'] = df_test['Spending_Score'].map(map_score)

In [17]:
df_train.sort_values(["Spending_Score"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Spending_Score"], axis=0, ascending=[True], inplace=True) 

In [18]:
#df_train["Graduated"].fillna( method ='ffill', inplace = True) 
#df_test["Graduated"].fillna( method ='ffill', inplace = True) 

In [19]:
df_train

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
4321,461253,2,1.0,18,No,Healthcare,0.0,1,5.0,Cat_4,D
7458,463312,1,2.0,40,No,Engineer,3.0,1,1.0,Cat_3,A
5865,463348,2,2.0,40,Yes,Doctor,1.0,1,2.0,Cat_6,A
3321,466806,1,2.0,40,Yes,Healthcare,0.0,1,1.0,Cat_6,D
2352,467070,1,2.0,40,No,Executive,5.0,1,4.0,Cat_6,D
...,...,...,...,...,...,...,...,...,...,...,...
2111,465416,1,2.0,55,No,Engineer,1.0,3,4.0,Cat_6,A
4130,463722,1,2.0,55,Yes,Executive,9.0,3,2.0,Cat_6,C
1116,462068,1,2.0,55,Yes,Executive,1.0,3,3.0,Cat_6,C
5121,467654,2,2.0,56,Yes,Artist,NaN,3,4.0,Cat_6,C


In [20]:
map_garduate={"No" : 1, "Yes": 2}
df_train['Graduated'] = df_train['Graduated'].map(map_garduate)
df_test['Graduated'] = df_test['Graduated'].map(map_garduate)

In [21]:
df_train['Profession'].value_counts()

Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: Profession, dtype: int64

In [22]:
map_profession={"Healthcare" : 1, "Engineer": 2,"Doctor":3,"Healthcare":4,"Executive":5}
df_train['Profession'] = df_train['Profession'].map(map_profession)
df_test['Profession'] = df_test['Profession'].map(map_profession)


In [23]:
df_train["Var_1"].value_counts()

Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: Var_1, dtype: int64

In [24]:
map_var={"Cat_1" : 1, "Cat_2": 2,"Cat_3":3,"Cat_4":4,"Cat_5":5,"Cat_6":6,"Cat_7":7}
df_train['Var_1'] = df_train['Var_1'].map(map_var)
df_test['Var_1'] = df_test['Var_1'].map(map_var)

In [25]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
4321,461253,2,1.0,18,1.0,4.0,0.0,1,5.0,4.0,D
7458,463312,1,2.0,40,1.0,2.0,3.0,1,1.0,3.0,A
5865,463348,2,2.0,40,2.0,3.0,1.0,1,2.0,6.0,A
3321,466806,1,2.0,40,2.0,4.0,0.0,1,1.0,6.0,D
2352,467070,1,2.0,40,1.0,5.0,5.0,1,4.0,6.0,D


In [26]:
df_train.corr()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
ID,1.000000,-0.006155,0.019420,-0.005055,-0.011544,-0.003726,-0.030688,0.005773,0.011801,-0.001116
Gender,-0.006155,1.000000,-0.114187,-0.030216,0.034109,-0.442594,0.059878,-0.061432,-0.055911,-0.030861
Ever_Married,0.019420,-0.114187,1.000000,0.568912,0.198102,0.025516,-0.095459,0.604278,-0.078956,0.084025
Age,-0.005055,-0.030216,0.568912,1.000000,0.237309,0.018807,-0.190789,0.415485,-0.280517,0.168308
Graduated,-0.011544,0.034109,0.198102,0.237309,1.000000,-0.029852,0.031374,0.103468,-0.231260,0.126698
Profession,-0.003726,-0.442594,0.025516,0.018807,-0.029852,1.000000,-0.021655,0.259112,0.156451,0.069117
Work_Experience,-0.030688,0.059878,-0.095459,-0.190789,0.031374,-0.021655,1.000000,-0.075194,-0.063234,0.023697
Spending_Score,0.005773,-0.061432,0.604278,0.415485,0.103468,0.259112,-0.075194,1.000000,0.094092,0.079258
Family_Size,0.011801,-0.055911,-0.078956,-0.280517,-0.231260,0.156451,-0.063234,0.094092,1.000000,-0.146823
Var_1,-0.001116,-0.030861,0.084025,0.168308,0.126698,0.069117,0.023697,0.079258,-0.146823,1.000000


In [27]:
df_train.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 

In [28]:
df_train["Graduated"].fillna( method ='ffill', inplace = True) 
df_test["Graduated"].fillna( method ='ffill', inplace = True) 

In [29]:
df_train.sort_values(["Gender"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Gender"], axis=0, ascending=[True], inplace=True)

In [30]:
df_train["Profession"].fillna( df_train['Profession'].mode()[0], inplace = True) 
df_test["Profession"].fillna( df_test['Profession'].mode()[0], inplace = True)

In [31]:
df_train.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 

In [32]:
df_train["Work_Experience"].fillna( df_train['Work_Experience'].mode()[0], inplace = True) 
df_test["Work_Experience"].fillna( df_test['Work_Experience'].mode()[0], inplace = True)


In [33]:
df_train["Family_Size"].fillna( df_train['Family_Size'].mode()[0], inplace = True) 
df_test["Family_Size"].fillna( df_test['Family_Size'].mode()[0], inplace = True)

In [34]:
df_train["Var_1"].fillna( df_train['Var_1'].mode()[0], inplace = True) 
df_test["Var_1"].fillna( df_test['Var_1'].mode()[0], inplace = True)

In [35]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8068 entries, 4321 to 4179
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   int64  
 2   Ever_Married     8068 non-null   float64
 3   Age              8068 non-null   int64  
 4   Graduated        8068 non-null   float64
 5   Profession       8068 non-null   float64
 6   Work_Experience  8068 non-null   float64
 7   Spending_Score   8068 non-null   int64  
 8   Family_Size      8068 non-null   float64
 9   Var_1            8068 non-null   float64
 10  Segmentation     8068 non-null   object 
dtypes: float64(6), int64(4), object(1)
memory usage: 756.4+ KB


In [36]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2627 entries, 1127 to 67
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2627 non-null   int64  
 1   Gender           2627 non-null   int64  
 2   Ever_Married     2627 non-null   float64
 3   Age              2627 non-null   int64  
 4   Graduated        2627 non-null   float64
 5   Profession       2627 non-null   float64
 6   Work_Experience  2627 non-null   float64
 7   Spending_Score   2627 non-null   int64  
 8   Family_Size      2627 non-null   float64
 9   Var_1            2627 non-null   float64
dtypes: float64(6), int64(4)
memory usage: 225.8 KB


In [37]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
4321,461253,2,1.0,18,1.0,4.0,0.0,1,5.0,4.0,D
5348,461312,1,1.0,18,1.0,4.0,3.0,1,3.0,2.0,D
6201,465354,1,1.0,18,1.0,3.0,0.0,1,5.0,2.0,C
1441,466846,1,1.0,18,1.0,4.0,2.0,1,4.0,2.0,D
4714,461279,1,1.0,18,1.0,3.0,4.0,1,2.0,6.0,A


In [38]:
df_train.sort_values(["ID"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["ID"], axis=0, ascending=[True], inplace=True) 

In [39]:
y_train=df_train['Segmentation']

In [40]:
x_train=df_train.drop(['Segmentation'],axis=1)
#df_test=df_test.drop(['ID'],axis=1)

In [41]:
y_train

7238    C
5546    C
4373    C
4695    D
3333    D
       ..
4791    C
570     D
5832    D
4140    A
4388    B
Name: Segmentation, Length: 8068, dtype: object

In [42]:
map={"A":1,"B":2,"C":3,"D":4}
y_train1 = y_train.map(map)


In [43]:
#x_train['Age']=(x_train['Age']-x_train['Age'].mean())/x_train['Age'].std()
x

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
5,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,C
8,465015,Female,Yes,61,Yes,Engineer,0.0,Low,3.0,Cat_7,D
12,461230,Female,No,19,No,Executive,0.0,Low,NaN,Cat_3,D
14,460849,Female,Yes,58,No,Doctor,0.0,Low,1.0,Cat_3,B
...,...,...,...,...,...,...,...,...,...,...,...
8060,463613,Female,Yes,48,Yes,Artist,0.0,Average,6.0,Cat_6,A
8061,465231,Male,Yes,65,No,Artist,0.0,Average,2.0,Cat_6,C
8062,463002,Male,Yes,41,Yes,Artist,0.0,High,5.0,Cat_6,B
8063,464018,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,D


In [44]:
#from xgboost import XGBClassifier

In [45]:
#XGBClassifier = XGBClassifier()


In [46]:
#XGBClassifier.fit(x_train,y_train)

In [47]:
#y_pred=XGBClassifier.predict(df_test)


In [48]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)

In [50]:
RandomForestClassifier =RandomForestClassifier()

In [51]:
RandomForestClassifier.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [52]:
y_pred=RandomForestClassifier.predict(df_test)

In [53]:
y_pred

array(['B', 'C', 'A', ..., 'A', 'B', 'A'], dtype=object)

In [54]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1,leaf_size=10,weights='distance',p=1,n_jobs=-1)

In [55]:
x_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
7238,458982,1,2.0,61,2.0,5.0,1.0,3,3.0,6.0
5546,458983,2,2.0,63,2.0,5.0,0.0,3,5.0,6.0
4373,458984,1,2.0,39,2.0,4.0,0.0,2,3.0,6.0
4695,458985,1,1.0,23,1.0,4.0,1.0,1,4.0,6.0
3333,458986,1,1.0,18,1.0,4.0,7.0,1,4.0,6.0


In [56]:
x_train['Famil/spending']=x_train['Spending_Score']*x_train['Family_Size']*x_train['Gender']
df_test['Famil/spending']=df_test['Spending_Score']*df_test['Family_Size']*x_train['Gender']

In [57]:
x_train['gender/marr']=x_train['Gender']*x_train['Ever_Married']
df_test['gender/marr']=df_test['Gender']*df_test['Ever_Married']

In [58]:
x_train=x_train.drop(['Work_Experience'],axis=1)
df_test=df_test.drop(['Work_Experience'],axis=1)

In [59]:
x_train['var1/id']=x_train['Var_1']*x_train['ID']
df_test['var1/id']=df_test['Var_1']*df_test['ID']


In [60]:
df_train['target']=y_train1
df_train.corr()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,target
ID,1.000000,-0.006155,0.019420,-0.005055,-0.013691,-0.010002,-0.018627,0.005773,0.012601,-0.001955,-0.009104
Gender,-0.006155,1.000000,-0.114187,-0.030216,0.033436,-0.259887,0.057212,-0.061432,-0.057920,-0.029660,-0.026403
Ever_Married,0.019420,-0.114187,1.000000,0.568912,0.199449,0.095532,-0.090410,0.604278,-0.069689,0.083725,-0.220317
Age,-0.005055,-0.030216,0.568912,1.000000,0.236881,0.121673,-0.177344,0.415485,-0.270793,0.167332,-0.243552
Graduated,-0.013691,0.033436,0.199449,0.236881,1.000000,0.076437,0.036285,0.104391,-0.218213,0.123487,-0.179263
Profession,-0.010002,-0.259887,0.095532,0.121673,0.076437,1.000000,-0.007441,0.185480,0.016935,0.071699,0.067891
Work_Experience,-0.018627,0.057212,-0.090410,-0.177344,0.036285,-0.007441,1.000000,-0.074266,-0.059565,0.025826,0.008216
Spending_Score,0.005773,-0.061432,0.604278,0.415485,0.104391,0.185480,-0.074266,1.000000,0.100744,0.078314,-0.109837
Family_Size,0.012601,-0.057920,-0.069689,-0.270793,-0.218213,0.016935,-0.059565,0.100744,1.000000,-0.137801,0.185486
Var_1,-0.001955,-0.029660,0.083725,0.167332,0.123487,0.071699,0.025826,0.078314,-0.137801,1.000000,-0.018339


In [61]:
neigh.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=1, p=1,
                     weights='distance')

In [62]:
y_pred=neigh.predict(df_test)

In [63]:
np.savetxt("data009.csv", y_pred,delimiter=',',fmt=['%s'])